<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from Akvo import Flow
from selenium import webdriver
from time import sleep
import pandas as pd
import urllib.parse
import requests as r
import json

In [2]:
instanceURL = "https://anu.akvolumen.org/api"
workdir = "../charts-generator/"

In [3]:
dboard = Flow.getData(instanceURL + '/library', Flow.getToken())
visual = dboard.get('visualisations')
df = pd.DataFrame(visual)
sl = pd.read_csv('list-of-figures.csv')
cm = sl[sl['source'] == 'Custom']
sl = sl[sl['source'] != 'Custom']

In [4]:
df['specstr'] = df['spec'].astype('str')

In [5]:
def parseString(x):
    x = x.replace("'",'"').replace('None','null').replace('False','false').replace('True','true')
    x = urllib.parse.unquote(x)
    x = urllib.parse.quote(x)
    return x

In [6]:
df['specurls'] = df['specstr'].apply(parseString)
df['endpoints'] = '/aggregation/' + df['datasetId'] + '/' + df['visualisationType'] + '?query=' + df['specurls']

In [7]:
df = df.merge(sl, left_on='id', right_on='source')

In [8]:
ds = df[['Fid','Fname','endpoints','visualisationType']]

In [9]:
ds = ds.rename(columns={
    'Fid':'id',
    'Fname':'name',
    'endpoints':'endpoint',
    'visualisationType':'type',
})

In [10]:
# Debug
# ds['endpoint'][1]
# testendpoint = '/aggregation/5c980e26-be25-47ee-9fe1-13eb6bdc2518/bar?query=%7B%22filters%22%3A%20%5B%5D%2C%20%22sort%22%3A%20null%2C%20%22subBucketColumn%22%3A%20%22c26020003%22%2C%20%22showLabels%22%3A%20false%2C%20%22metricAggregation%22%3A%20%22count%22%2C%20%22axisLabelXFromUser%22%3A%20false%2C%20%22legendTitle%22%3A%20%22%285.1.26%29%20Do%20you%20think%20that%20the%20winning%20candidate%20will%20be%20able%20to%20find%20out%20afterwards%20who%20you%20voted%20for%3F%22%2C%20%22axisLabelY%22%3A%20%22%20-%20count%22%2C%20%22horizontal%22%3A%20false%2C%20%22version%22%3A%201%2C%20%22metricColumnX%22%3A%20null%2C%20%22bucketColumn%22%3A%20%22d2%22%2C%20%22metricColumnY%22%3A%20null%2C%20%22showValueLabels%22%3A%20true%2C%20%22legendPosition%22%3A%20%22top%22%2C%20%22colors%22%3A%20%7B%22No%22%3A%20%22%23bf360c%22%2C%20%22Yes%22%3A%20%22%23388e3c%22%2C%20%22Not%20Sure%22%3A%20%22%23795548%22%7D%2C%20%22subBucketMethod%22%3A%20%22stack_percentage%22%2C%20%22axisLabelX%22%3A%20%22constituency-2%22%2C%20%22truncateSize%22%3A%20null%2C%20%22axisLabelYFromUser%22%3A%20false%7D'
# Flow.getData(instanceURL + testendpoint, Flow.getToken())

In [11]:
getLegend = lambda x : [y['key'] for y in x]
getLabel = lambda x : [y['label'] for y in x]
getData = lambda x : [y['value'] for y in x]
ordered_province = pd.DataFrame([
    {'ordered':'Central Makira'},
    {'ordered':'East Guadalcanal'},
    {'ordered':'Central Guadalcanal'},
    {'ordered':'East Honiara'},
    {'ordered':'Central Honiara'},
    {'ordered':'West Honiara'},
    {'ordered':'Aoke Langalanga'},
    {'ordered':'Small Malaita'},
    {'ordered':'Lau Mbaelelea'},
    {'ordered':'Russell Savo'},
    {'ordered':'Gao Bugotu'},
    {'ordered':'Maringe Kokota'},
    {'ordered':'Gizo Kolombangara'},
    {'ordered':'North Vella Lavella'},
    {'ordered':'North West Choiseul'}
])
ordered_province.reset_index(level=0, inplace=True)

In [12]:
def checkPercentage(allValues):
    index = 0
    diff = 0
    total = sum(allValues)
    if total < 100:
        index = allValues.index(min(allValues))
        diff = 100 - total
    elif sum(allValues) > 100:
        index = allValues.index(max(allValues))
        diff = 100 - total
    else:
        pass
    return [index,diff]

In [13]:
def createBarChart(b,ids):
    ordered = ordered_province
    data = b
    output = r.get('http://localhost:4000/draft/results-barchart.json').json()
    unordered = pd.DataFrame(data['common']['data'])
    unordered.reset_index(level=0, inplace=True)
    ordered_index = unordered.merge(ordered,left_on='key',right_on='ordered')
    ordered_index = ordered_index.sort_values(by=['index_y'], ascending=True)
    ordered_index = list(ordered_index['index_x'])
    output['legend']['data'] = getLegend(data['series'])
    output['xAxis']['data'] = list(ordered['ordered'])
    for i,l in enumerate(output['legend']['data']):
        if i == 0:
            df = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})
        else:
            df[i] = pd.DataFrame(data['series'][i]['data']).rename(columns={'value':i})[i]
    df['sum'] = df.sum(axis = 1, skipna = True)
    df['order'] = pd.Series(ordered_index)
    df = df.sort_values(by=['order'], ascending=True).reset_index()
    for i,l in enumerate(output['legend']['data']):
        df[i] = round(df[i] / df['sum'] * 100).astype(int)
    df = df.drop(columns=['sum'])
    series = list(range(len(output["legend"]["data"])))
    for i, rows in df.iterrows():
        new_list = []
        for a in series:
            new_list.append(rows[a])
        adjust = checkPercentage(new_list)
        df[adjust[0]][i] = df[adjust[0]][i] + adjust[1]
    output['series'] = []
    for i,l in enumerate(output["legend"]["data"]):
        output["series"].append({"name": l,
                                 "type": "bar",
                                 "stack": "total",
                                 "label": {
                                     "normal": {
                                         "fontSize": 20,
                                         "fontWeight": "bold",
                                         "show": True,
                                         "position": "inside",
                                         "formatter": "{c}%"
                                     }
                                },"data": list(df[i])})
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + "json-source/" + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [14]:
def createPieChart(b,ids):
    data = b
    output = r.get('http://localhost:4000/draft/results-piechart.json').json()
    legend = getLegend(data['common']['data'])
    output['title']['text'] = str(sum(getData(data['series'][0]['data'])))
    output['series'][0]['data'] = []
    for i,d in enumerate(data['series'][0]['data']):
        output['series'][0]['data'].append({'value':d['value'],'name':legend[i]})
    json_output = json.dumps(output)
    jname = ids.replace(".","-")
    f = open(workdir + "json-source/" + jname +".json","w")
    f.write(json_output)
    f.close()
    return jname

In [15]:
def generateChart(endpoint,charttype,ids):
    data = Flow.getData(instanceURL + endpoint, Flow.getToken())
    if charttype == 'pie':
        result = createPieChart(data,ids)
    else:
        result = createBarChart(data,ids)
    return result

In [16]:
# Debug
ds = ds.iloc[:2]

In [17]:
ds['json'] =  ds.apply(lambda x: generateChart(x['endpoint'],x['type'],x['id']), axis=1)

In [18]:
def takeScreenShot(x,y):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('http://localhost:4000/#'+x)
    sleep(2)
    imagename = x + '-' + y.lower().replace('?',' ').replace('(','').replace(')','').replace(' ','-')
    driver.save_screenshot('screenshots/'+ imagename +'.png')
    driver.close()
    return imagename + '.png'

In [19]:
ds['img'] = ds.apply(lambda x: takeScreenShot(x['json'],x['name']),axis=1)